**Author:** A.S. Grm (aleksander.grm@fpp.uni-lj.si)

**Date:** 2024

<hr>

# Star Identification

The process of star identification involves using the [Nautical Almanac](https://www.thenauticalalmanac.com) (*Dec* and *Sha* stars) and measuring the altitude $H$ and azimuth $\omega$ of the star.

In addition to the measurement itself, we also need to determine the measurement time and location. Positions are entered in the format commonly used in navigation:

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$,

where degrees are given as whole numbers and minutes to one decimal place precision.

Input format for position:
- LAT-$\varphi$ and LONG-$\lambda$ are entered in the format [degrees, minutes, cardinal direction]

Example input for a position:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- The cardinal direction must be in **single quotes**, with permissible characters 'N', 'S', 'E', 'W'

Measurement time is recorded as:
- date: *date*=[day,month,year]
- time: *time*=[hour,minutes, seconds]

Altitude and azimuth are recorded as:
- altitude: $h=\#\#^\circ \#\#'$=[deg,min]
- azimuth: $\omega=\#\#\#^\circ$=[deg]

<hr>

**Star Identification Procedure**

Let's determine the input parameters:

1. determine assumed position $P_0$.,
2. determine the measurement time (*date*, *time*),
3. measure the altitude of the celestial body using a sextant $H_\text{s}$,
4. correct measured altitude for altitude errors and obtaon observer altitude $H_\text{o}$,
5. measure the azimuth to the celestial body $\omega$.

Now we proceed with calculations in the following steps:
1. Convert the measured quantities:
\begin{align*}
    Z & = 90^\circ - H \\
    \psi & = 90^\circ - \varphi
\end{align*}

2. Calculate the distance $p$ and then the declination $\delta$ of the measured star:
\begin{align*}
    \cos p & = \cos Z \cos \psi + \sin Z \sin \psi \cos \omega \\
    \delta & = 90^\circ - p
\end{align*}

3. Determine the local hour angle of the star $\mathrm{Lha}$:
$$
\cos \mathrm{Lha} = \frac{\cos Z - \cos p \cos \psi}{\sin p \sin \psi}
$$
where it is necessary to adjust the calculated $\mathrm{Lha}$ if $\omega < 180^\circ$:
$$
\mathrm{Lha} = 360^\circ - \mathrm{Lha}
$$

4. Using $\mathrm{Lha}$, we can determine the sidereal hour angle of the star $\mathrm{Sha}$:
\begin{align*}
    \mathrm{Lha}_{\gamma} & = \mathrm{Gha}_{\gamma} + (\pm \lambda) \\
    \mathrm{Sha} & = \mathrm{Lha} - \mathrm{Lha}_{\gamma}
\end{align*}
where if $\mathrm{Sha} < 0^\circ$, we need to adjust the calculated $\mathrm{Sha}$ as follows:
$$
\mathrm{Sha} = 360^\circ + \mathrm{Sha}
$$

Now that we have determined the declination of the star $\delta$ and its sidereal hour angle $\mathrm{Sha}$, we can identify the star's name using the *Nautical Almanac*.

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
# Finds all visible celestial bodies above horizon
def findAllVisibleStars(t,pos,ns_db):
    
    star_ids = ns_db.keys()
    h0 = 0
    h1 = 90
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'dec':sdd['dec'], 'sha':sdd['sha'], 'alt':saz['alt'], 'az':saz['az']})
    
    return vs

In [ ]:
# Find star dec and sha
def getStarDecAndSha(s_h,s_w,pos,date,time):
    
    t = [date[0],date[1],date[2],time[0],time[1],time[2]]
    msg = ''
 
    gha_a = cd.get_aries_gha(t)
    lha_a = gha_a + pos[1]

    s_z = nt.deg2rad(90.0 - s_h)
    s_psi = nt.deg2rad(90.0 - pos[0])

    # find declination
    cos_s_p = mat.cos(s_z)*mat.cos(s_psi) + mat.sin(s_z)*mat.sin(s_psi)*mat.cos(nt.deg2rad(s_w))
    if mat.fabs(cos_s_p) > 1.0: msg='cos_p > 1.0 ({:.5f})'.format(cos_s_p)
    if cos_s_p > 1.0:
        cos_s_p = 1.0
    if cos_s_p < -1.0:
        cos_s_p = -1.0
    s_p = mat.acos(cos_s_p) 
    dec = 90.0 - nt.rad2deg(s_p)

    # find local hour angle
    # !! due to measured data it can be out of range (out of interval [-1,1]) !!!
    cos_lha = (mat.cos(s_z) - mat.cos(s_p)*mat.cos(s_psi))/(mat.sin(s_p)*mat.sin(s_psi))
    if mat.fabs(cos_lha) > 1.0: 
        if msg == '':
            msg='cos_lha > 1.0 ({:.5f})'.format(cos_lha)
        else:
            msg = msg + ' and cos_lha > 1.0 ({:.5f})'.format(cos_lha)
    if cos_lha > 1.0:
        cos_lha = 1.0
    if cos_lha < -1.0:
        cos_lha = -1.0
        
    lha = nt.rad2deg(mat.acos(cos_lha))
    if s_w < 180.0:
        lha = 360 - lha

    # find siderial hour angle
    sha = lha - lha_a
    if sha < 0:
        sha = 360 + sha
        
    return [dec,sha,msg]

In [ ]:
# Finds star name
def findStarName(ns_db,pos,date,time,dec,sha):
    
    t = [date[0],date[1],date[2],time[0],time[1],time[2]]
    
    vs = findAllVisibleStars(t,pos,ns_db)
    
    dist = []
    stars = []
    for s in vs:
        dd = s['dec'] - dec
        ds = s['sha'] - sha
        da = mat.fabs(dd) + mat.fabs(ds)
        dist.append(da)
        stars.append(s)

        #print('name: {:12s}, dist: {:6.2f}, dec: {:5.2f}, sha: {:5.2f}'.format(
        #    s['name'],da,s['dec'],s['sha']))

    # find the star with minimal dec+sha distance
    min_dist = min(dist)
    min_idx = dist.index(min_dist)
    ac = stars[min_idx]
    
    return ac

In [ ]:
# load Astro database
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

In [ ]:
# Main program

# Date & time
date = [2022,2,21] # [yyyy, mm, dd]
time = [19,0,0]    # [HH,MM,SS] in UTC

# Apparent position
fi = [46,6.08,'N']  # Latitude
la = [13,38.85,'E'] # Longitude
h = 20.0 # observer height in [m]

# Star measurements
s_h = [51, 10] # observed height [deg, min.dec]
s_w = 182.0    # observed azimuth in [deg]


pos = [nt.nav2dd(fi), nt.nav2dd(la), h]
s_h = nt.dms2dd(s_h)

print('MEASURED data:')
print('  -> alt: {:s}'.format(nt.prettyPrintAlt(s_h)))
print('  ->  az: {:s}'.format(nt.prettyPrintAz(s_w)))

print()
[dec, sha, msg] = getStarDecAndSha(s_h, s_w, pos, date, time)
if msg == '':
    print('CALCULATED data:')
else:
    print('CALCULATED data (*WARNING* {:s}):'.format(msg))
print('  -> dec:  {:s}'.format(nt.prettyPrintDec(dec)))
print('  -> sha: {:s}'.format(nt.prettyPrintHA(sha)))
    
ss = findStarName(ns_db,pos,date,time,dec,sha) 

print()
print('Identification with cloasest star from ALMANACH:')
print('  -> name: {:s}'.format(ss['name'].upper()))
print('  ->  dec:  {:s}'.format(nt.prettyPrintDec(ss['dec'])))
print('  ->  sha: {:s}'.format(nt.prettyPrintHA(ss['sha'])))
print('  ->  alt:  {:s}'.format(nt.prettyPrintAlt(ss['alt'])))
print('  ->   az: {:s}'.format(nt.prettyPrintAz(ss['az'])))